## Test Environment for completing task 3

In [1]:
from elasticsearch import Elasticsearch, exceptions
from tokenizer import porter_processing

es = Elasticsearch([{"host":"localhost","port":9200, "scheme": "http"}])
print(f"Elasticsearch instantiated? {es.ping()}")

Elasticsearch instantiated? True


### (DNE) Getting mtermvector data

In [ ]:
response = es.mtermvectors(index="ap89",term_statistics= True, ids = ["AP890222-0173","AP890222-0174"], fields = "_content")

import json

file_path = 'mtermvectors_response_coupled.json'

with open(file_path, 'w') as file:
    json.dump(response, file, indent=2)

### Processing the query

In [5]:
import json

In [6]:
from tokenizer import porter_processing

query_processed = porter_processing("identify individuals or organizations which produce fiber optics equipment.","smh")

query = {
  "query": {
    "match": {
      "_content": query_processed
    }
  }
}

# Replace 'your_index_name' with the name of your index
response = es.search(index="ap89", body=query)

file_path = 'queryresp_mod.json'

with open(file_path, 'w') as file:
    json.dump(response, file, indent=2)
print(response)
# Print the returned documents
# for hit in response['hits']['hits']:
#     print(hit['_source'])  # Prints out the source of each hit

Tokenization started for smh
Tokenization completed for smh
{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 26.04061, 'hits': [{'_index': 'ap89', '_type': '_doc', '_id': 'AP890110-0294', '_score': 26.04061, '_source': {'_content': 'intern busi machin corp buy 25 percent fiberopt compani own corn glass work compani tuesday signal grow import fiber comput world valu deal disclos pco 175employe compani base chatsworth area lo angel special devic convert electron signal light vice versa fiber optic mainli longdist telephon line ibm invest signal compani idea speed permit fiber optic use comput field spokesman paul bergevin today corn base corn ny made addit invest pco keep 65 percent interest compani remain 10 percent own pco employe founder individu wire comput user begin connect cluster person comput workstat highcapac fiberopt cabl john kessler presid kessler market i

/tmp/ipykernel_72727/909527558.py:14: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(index="ap89", body=query)


In [11]:
response['hits']['hits']

[{'_index': 'ap89',
  '_type': '_doc',
  '_id': 'AP890110-0294',
  '_score': 26.04061,
  '_source': {'_content': 'intern busi machin corp buy 25 percent fiberopt compani own corn glass work compani tuesday signal grow import fiber comput world valu deal disclos pco 175employe compani base chatsworth area lo angel special devic convert electron signal light vice versa fiber optic mainli longdist telephon line ibm invest signal compani idea speed permit fiber optic use comput field spokesman paul bergevin today corn base corn ny made addit invest pco keep 65 percent interest compani remain 10 percent own pco employe founder individu wire comput user begin connect cluster person comput workstat highcapac fiberopt cabl john kessler presid kessler market intellig newport ri market grew 100 percent 1987 1988 continu heavi growth five year kessler longerrang goal fiber optic comput connect chip light signal chip pco probabl wellequip area kessler'}},
 {'_index': 'ap89',
  '_type': '_doc',
  '

### Query Processing fetched from reference

In [12]:
QUERY_PATH = "/home/burpcat/Documents/assignments/ir/hw1-burpcat/IR_data/AP_DATA/query_desc.51-100.short.txt"

def load_queries():
    queries = []
    with open(QUERY_PATH) as f:
        lines = [line.rstrip() for line in f]
        for query in lines:
            if (query):
                id = int(query.split('.')[0])
                content = query.split('.')[1].strip()
                queries.append((id, porter_processing(content,"test").split()))
    return queries

queries = load_queries()
query_ids = [q[0] for q in queries]
print(queries)

Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokenization started for test
Tokenization completed for test
Tokeniza

In [ ]:
print(queries[0])

### Term Vector Processing

In [13]:
# Getting a list of all the documents
import re

def documentid_fetcher():

    document_list = []
    file_path = "/home/burpcat/Documents/assignments/ir/hw1-burpcat/IR_data/AP_DATA/doclist_new_0609.txt"

    with open(file_path, 'r') as file:
        for line in file:
            document_list.append(line.split()[1])
            
        return document_list

In [14]:
document_list = documentid_fetcher()

def get_term_vectors_batch(index_name, document_list, batch_size):
    responses = []  # List to store responses
    for i in range(0, len(document_list), batch_size):
        batch = document_list[i:i + batch_size]
        print(batch)
        response = es.mtermvectors(index=index_name, term_statistics=True, ids=batch, fields=["_content"])
        responses.append(response)
    return responses

all_responses = get_term_vectors_batch("ap89", document_list, batch_size=1)

['AP891231-0001']
['AP890930-0001']
['AP890930-0002']
['AP890930-0003']
['AP890930-0004']
['AP890930-0005']
['AP890930-0006']
['AP890930-0007']
['AP890930-0008']
['AP890930-0009']
['AP890930-0010']
['AP890930-0011']
['AP890930-0012']
['AP890930-0013']
['AP890930-0014']
['AP890930-0015']
['AP890930-0016']
['AP890930-0017']
['AP890930-0018']
['AP890930-0019']
['AP890930-0020']
['AP890930-0021']
['AP890930-0022']
['AP890930-0023']
['AP890930-0024']
['AP890930-0025']
['AP890930-0026']
['AP890930-0027']
['AP890930-0028']
['AP890930-0029']
['AP890930-0030']
['AP890930-0031']
['AP890930-0032']
['AP890930-0033']
['AP890930-0034']
['AP890930-0035']
['AP890930-0036']
['AP890930-0037']
['AP890930-0038']
['AP890930-0039']
['AP890930-0040']
['AP890930-0041']
['AP890930-0042']
['AP890930-0043']
['AP890930-0044']
['AP890930-0045']
['AP890930-0046']
['AP890930-0047']
['AP890930-0048']
['AP890930-0049']
['AP890930-0050']
['AP890930-0051']
['AP890930-0052']
['AP890930-0053']
['AP890930-0054']
['AP890930

In [15]:
len(all_responses)

84678

In [16]:
def extracting_termvectors(all_responses):
    number_of_documents = len(document_list)
    documents_and_vectors = {}
    documents_and_documentlength = {}
    term_to_docfrequency = {}
    termtotalfrequency = {}
    terms_list = set()
    missed_docs = []
    # global documents_and_documentlength  # If you intend to modify a global variable, declare it (though it's better to avoid global variables when possible)
    
    for ele in all_responses:
        term_vector = ele['docs']
        document_id = term_vector[0]["_id"]

        if "_content" not in term_vector[0]["term_vectors"]:
            documents_and_vectors[document_id] = {}
            documents_and_documentlength[document_id] = 0  # Assuming you want to record a length of 0 for documents without "_content"
            missed_docs.append(document_id)
        else:
            terms = term_vector[0]["term_vectors"]["_content"]["terms"]

            
            for term, details in terms.items():
                term_to_docfrequency[term] = details['doc_freq']
                termtotalfrequency[term] = details['ttf']
                terms_list.add(term)  # Add each term within the loop

            documents_and_vectors[document_id] = terms
            documents_and_documentlength[document_id] = sum(details['term_freq'] for details in terms.values())
        
    return number_of_documents,documents_and_vectors,documents_and_documentlength,term_to_docfrequency,termtotalfrequency,terms_list,missed_docs

In [17]:
number_of_documents,documents_and_vectors,documents_and_documentlength,term_to_docfrequency,termtotalfrequency,terms_list,missed_docs = extracting_termvectors(all_responses)

In [18]:
avg_doc_length = sum([documents_and_documentlength[ele] for ele in document_list])  / number_of_documents
total_doc_length = sum([documents_and_documentlength[ele] for ele in documents_and_documentlength])
vocab_size = len(terms_list)

In [19]:
# Return document frequency for a term

def documentfrequency(term):
    if term in term_to_docfrequency:
        return term_to_docfrequency[term]
    else:
        return 0

# return term frequency for a term
def termfrequency(term,document_id):
    if term in documents_and_vectors[document_id]:
        return documents_and_vectors[document_id][term]['term_freq']
    else:
        return 0
    
# return term frequency in a query
def termfrequency_in_query(term,query):
    return query.count(term)

### (DNE) Generating a score list to store scores of each retreival model for a query

In [20]:
from pprint import pprint

models = ['es', 'okapi', 'tfidf', 'bm25', 'lm_laplace', 'lm_jm']


''' Store scores of all the retrieval models. Map the name of the model
    to a map of query id to a score (q_id, doc_id, score).
'''
#old technique
# query_scores = { m: {q: [] for q in query_ids} for m in models}

#new technique
query_scores = { m : {q : { d : "" for d in document_list} for q in query_ids} for m in models}

In [21]:
# testing whether it exists or not
query_scores['okapi'][85]['AP891231-0001']

''

In [22]:
# Okapi TF model

def okapi_tf(w, d):
    return termfrequency(w,d) / \
    (termfrequency(w,d) + 0.5 + (1.5 * (documents_and_documentlength[d] / avg_doc_length)))

''' 1st vector space model using modified version of TF, accounting for
    document length.
'''
def okapi_score(d, q):
    return sum([okapi_tf(w, d) for w in q])

In [ ]:
type(documents_and_documentlength)

In [ ]:
len(queries)

In [ ]:
# for multiple models

rand_list = []
for model, query_scores in query_scores.items():

    for query in queries:
        query_id = query[0]
        query_tokens = query[1]

        rand_list.append(f'q{query_id}')
        for doc_id in document_list:
            
            if documents_and_documentlength[doc_id] <= 0:
                continue

            scores = okapi_score(doc_id,query_tokens)
            query_scores

In [ ]:
### DO NOT EXECUTE

for query in queries:
    query_id = query[0]
    query_tokens = query[1]

    rand_list.append(f'q{query_id}')
    for doc_id in document_list:

        print(query_id,doc_id)        
        query_doc_scores = []
        
        if documents_and_documentlength[doc_id] <= 0:
            continue

        scores = okapi_score(doc_id,query_tokens)
        query_doc_scores.append(str(QueryScore(query_id, doc_id, scores)))
        
    query_doc_scores = sorted(query_doc_scores, reverse=True)[:1000]
        
    query_scores['okapi'][query_id] = query_doc_scores

In [23]:
rand_list = []
crank_list = []
for query in queries:
    query_id = query[0]
    query_tokens = query[1]
    
    for doc_id in document_list:

        if not documents_and_documentlength[doc_id] > 0:
            crank_list.append([query,doc_id])
            query_scores['okapi'][query_id][doc_id] = "0"
        else:
            rand_list.append([query,doc_id])
            scores = okapi_score(doc_id,query_tokens)
            query_scores['okapi'][query_id][doc_id] = str(scores)
    
for query_id, docs_scores in query_scores['okapi'].items():
    # Convert the dictionary to a list of tuples [(doc_id, score), ...] and sort it
    sorted_scores = sorted(docs_scores.items(), key=lambda x: float(x[1]), reverse=True)[:1000]

    # Re-construct the dictionary for this query_id with sorted and trimmed scores
    query_scores['okapi'][query_id] = {doc_id: score for doc_id, score in sorted_scores}

In [ ]:
model_name = "okapi"

print(f"Writing output to file for {model_name}")
filename = f"{model_name}_output.txt"
with open(filename, 'w') as file:
    for ele in query_ids:
        print(model_name,ele)
        rank = 0
        data_to_write = query_scores[model_name][ele]
        for doc_id, score in data_to_write.items():
            rank+=1
            # <query-number> Q0 <docno> <rank> <score> Exp
            file.write(f"{ele} Q0 {doc_id} {rank} {score} Exp\n")


In [32]:
for ele in queries():
    print(ele)

TypeError: 'list' object is not callable

In [47]:
model = "es"

for query in queries:
    query_id = query[0]
    query_tokens = query[1]
    
    if model=="es":
        es_query = {
            "query": {
                "match": {
                "_content": ' '.join(query_tokens)
                }
            },
            "size" : 1000
        }

        response = es.search(index="ap89", body=es_query)

        for hit in response['hits']['hits']:
            es_docno = hit["_id"]
            es_score = hit["_score"]
            print(f"{es_docno}-{es_score}")
            if es_score != 0:
                print(es_score)
                query_scores['es'][query_id][es_docno]= str(es_score)

/tmp/ipykernel_72727/2078152760.py:17: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(index="ap89", body=es_query)


AP890220-0143-19.41699
19.41699
AP890221-0159-18.993114
18.993114
AP890206-0065-18.930449
18.930449
AP890129-0035-18.596272
18.596272
AP891205-0128-18.562588
18.562588
AP891225-0110-18.343376
18.343376
AP890915-0075-18.284477
18.284477
AP890515-0006-18.087315
18.087315
AP891011-0056-17.865993
17.865993
AP890123-0065-17.848095
17.848095
AP890527-0048-17.833515
17.833515
AP891026-0155-17.69399
17.69399
AP890724-0073-17.61335
17.61335
AP890509-0275-17.554096
17.554096
AP890721-0159-17.29088
17.29088
AP890301-0215-17.275948
17.275948
AP890628-0156-17.153406
17.153406
AP890401-0001-17.110802
17.110802
AP890517-0280-17.109585
17.109585
AP890412-0196-16.999113
16.999113
AP890125-0265-16.86194
16.86194
AP891205-0163-16.828592
16.828592
AP890228-0162-16.699312
16.699312
AP890314-0022-16.686485
16.686485
AP890314-0223-16.593584
16.593584
AP890713-0174-16.356794
16.356794
AP890720-0157-16.354065
16.354065
AP891221-0133-16.30709
16.30709
AP890125-0007-16.233234
16.233234
AP890808-0004-16.100431
16

In [55]:
with open("es_out_raw.txt", 'w') as file:
    file.write(str(query_scores['es']))

In [48]:
query_scores['es'][85]

{'AP890504-0175': '9.764589',
 'AP890220-0143': '19.41699',
 'AP890221-0159': '18.993114',
 'AP890206-0065': '18.930449',
 'AP890129-0035': '18.596272',
 'AP891205-0128': '18.562588',
 'AP891225-0110': '18.343376',
 'AP890915-0075': '18.284477',
 'AP890515-0006': '18.087315',
 'AP891011-0056': '17.865993',
 'AP890123-0065': '17.848095',
 'AP890527-0048': '17.833515',
 'AP891026-0155': '17.69399',
 'AP890724-0073': '17.61335',
 'AP890509-0275': '17.554096',
 'AP890721-0159': '17.29088',
 'AP890301-0215': '17.275948',
 'AP890628-0156': '17.153406',
 'AP890401-0001': '17.110802',
 'AP890517-0280': '17.109585',
 'AP890412-0196': '16.999113',
 'AP890125-0265': '16.86194',
 'AP891205-0163': '16.828592',
 'AP890228-0162': '16.699312',
 'AP890314-0022': '16.686485',
 'AP890314-0223': '16.593584',
 'AP890713-0174': '16.356794',
 'AP890720-0157': '16.354065',
 'AP891221-0133': '16.30709',
 'AP890125-0007': '16.233234',
 'AP890808-0004': '16.100431',
 'AP891027-0091': '15.982824',
 'AP890825-0032

In [46]:
query_scores['okapi'][85]

{'AP890220-0143': '3.1710097817594836',
 'AP890221-0159': '2.8652769974676913',
 'AP890915-0075': '2.768924937411609',
 'AP891101-0036': '2.7156086500238783',
 'AP891221-0133': '2.6997743185745',
 'AP890129-0035': '2.644868269922066',
 'AP890412-0196': '2.632960980724519',
 'AP891031-0163': '2.6293535392311633',
 'AP890417-0158': '2.616248184853358',
 'AP890818-0018': '2.6115703394336207',
 'AP890302-0132': '2.604311909185586',
 'AP890807-0158': '2.6042776999437507',
 'AP891225-0110': '2.559007921942075',
 'AP890131-0031': '2.5585992442145455',
 'AP890518-0050': '2.5429929161583766',
 'AP890516-0104': '2.5178875976975332',
 'AP891111-0095': '2.512428961174475',
 'AP890823-0148': '2.506167178268911',
 'AP890731-0042': '2.4956505700263847',
 'AP890223-0103': '2.4868719808854403',
 'AP891012-0293': '2.485453262783678',
 'AP890209-0040': '2.47944020470985',
 'AP890919-0162': '2.4768779213723655',
 'AP890825-0032': '2.4737928604496675',
 'AP891109-0225': '2.4515573757579885',
 'AP890125-000

In [50]:
for query_id, docs_scores in query_scores["es"].items():
    # Convert the dictionary to a list of tuples [(doc_id, score), ...] and sort it
    sorted_scores = sorted(docs_scores.items(), key=lambda x: float(x[1]), reverse=True)[:1000]

    # Re-construct the dictionary for this query_id with sorted and trimmed scores
    query_scores[model][query_id] = {doc_id: score for doc_id, score in sorted_scores}

### Writing queries to files, perfect function

In [51]:
def filewriter(model_name,query_ids,query_scores):

    filename = f"{model_name}_output_2.txt"
    with open(filename, 'w') as file:
        for ele in query_ids:
            print(model_name,ele)
            rank = 0
            data_to_write = query_scores[model_name][ele]
            for doc_id, score in data_to_write.items():
                rank+=1
                # <query-number> Q0 <docno> <rank> <score> Exp
                file.write(f"{ele} Q0 {doc_id} {rank} {score} Exp\n")

In [52]:
filewriter(model_name="es",query_ids=query_ids,query_scores=query_scores)

es 85
es 59
es 56
es 71
es 64
es 62
es 93
es 99
es 58
es 77
es 54
es 87
es 94
es 100
es 89
es 61
es 95
es 68
es 57
es 97
es 98
es 60
es 80
es 63
es 91


In [ ]:
some_list = ["model1", 'model2']

def model1():
    print("hello model1")

def model2():
    print("hello model2")

codec = "model1"

def model_handler():
    codec()

In [ ]:
model_handler()